In [2]:
import sqlite3
import pandas as pd

# 1) Crear conexión a un archivo .db
# En este punto se crea el archivo 'suministros.db' en tu carpeta de trabajo si no existe.
conexion = sqlite3.connect("suministros.db")
cursor = conexion.cursor()

# 2) Crear tablas con sus relaciones
# CREATE TABLE-> Es para crear una tabla en la base de datos
# IF NOT EXIST proveedores, nos evita errores si la tabla ya esta creada en este caso proveedrores
# Luego ya introducimos cada de las columnas con los titulos que queremos que tenga, tambein implementamos not null para no permitir que esas variables esten vacias
# Tambien ponemos el tipo de variable si es integer y tambien la primary key y la foreign key.
cursor.execute("""
CREATE TABLE IF NOT EXISTS Proveedor (
    proveedor_id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    direccion TEXT,
    ciudad TEXT,
    provincia TEXT)""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Categoria (
    categoria_id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL)""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Pieza (
    pieza_id INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    color TEXT,
    precio REAL NOT NULL,
    categoria_id INTEGER,
    FOREIGN KEY (categoria_id) REFERENCES Categoria(categoria_id))""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Suministro (
    suministro_id INTEGER PRIMARY KEY,
    proveedor_id INTEGER,
    pieza_id INTEGER,
    cantidad INTEGER NOT NULL,
    fecha DATE NOT NULL,
    FOREIGN KEY (proveedor_id) REFERENCES Proveedor(proveedor_id),
    FOREIGN KEY (pieza_id) REFERENCES Pieza(pieza_id))""")

#commit sirve para guardar los cambios que hayamos hehco de forma defintiva en la base de datos
conexion.commit()

# 3)Insertamos datos de ejemplo
#Aqui ejecuto executemany porque quiero insertar varios registros a la vez aunque solo son dos tuplas lo que le metemos y pongo las interogaciones son para rellenarse de las tuplas que introducimos abajo
cursor.executemany("""
INSERT INTO Proveedor (proveedor_id, nombre, direccion, ciudad, provincia)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, "Proveedor A", "Calle Bartolo 1", "Madrid", "Madrid"),
    (2, "Proveedor B", "Av. Segismundo 45", "Barcelona", "Cataluña")])

cursor.executemany("""
INSERT INTO Categoria (categoria_id, nombre)
VALUES (?, ?)
""", [
    (1, "Electrónica"),
    (2, "Mecánica")])

cursor.executemany("""
INSERT INTO Pieza (pieza_id, nombre, color, precio, categoria_id)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, "Resistor", "Rojo", 0.10, 1),
    (2, "Tornillo", "Plateado", 0.05, 2)])

cursor.executemany("""
INSERT INTO Suministro (suministro_id, proveedor_id, pieza_id, cantidad, fecha)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, 1, 1, 1000, "2025-11-01"),
    (2, 2, 2, 500, "2025-11-15"),
    (3, 1, 2, 200, "2025-11-20")])

conexion.commit()


IntegrityError: UNIQUE constraint failed: Proveedor.proveedor_id

- Para hacer pruebas hago un consulta a la base de datos en SQL:


In [ ]:
#Introducimos la funcion de Jaime:

# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas como nos dice Jaime en los videos 
def sql_query(query):

    # Ejecuta la query
    cursor.execute(query) # Recuerda que sólo funcionará si has llamado cursor
                                    # a tu cursor, si no, cambia el nombre en todo el código de la función

    # Almacena los datos de la query(fetchall() devuelve todas las filas de la consulta en una lista de tuplas)
    ans = cursor.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor.description]

    return pd.DataFrame(ans,columns=names)


# 4)Consultamos datos para verificar
query="""
SELECT s.suministro_id, pr.nombre AS proveedor, pi.nombre AS pieza, s.cantidad, s.fecha
FROM Suministro s
JOIN Proveedor pr ON s.proveedor_id = pr.proveedor_id
JOIN Pieza pi ON s.pieza_id = pi.pieza_id"""

sql_query(query)

# 5)Cerrarmos conexión
#conexion.close()

,proveedor_id,nombre,direccion,ciudad,provincia
0,1,Proveedor A,Calle Bartolo 1,Madrid,Madrid
1,2,Proveedor B,Av. Segismundo 45,Barcelona,Cataluña
